In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import logging

logging.basicConfig(filename='log.log', filemode='a', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
pd.set_option('display.max_columns', None)


In [ ]:
def popZeroID(row):
    x = row['TitleID'].split(".")
    return(x[0])


def mapTitlebyTitleID(row):
    title_id = row['TitleID']
    title = mapping.loc[mapping['IDTitle'] == title_id].Title
    return(title.to_list()[0])

def mapBinding(row):
    isbn = row['ISBN']
    bind_type = binding_mapping_s22.loc[binding_mapping_s22['BINDING::ISBNBindLineExport'] == isbn]['BINDING::AbbrBind']
    return(bind_type.to_list()[0])
    
def mapPrice(row):
    isbn = row['ISBN']
    try:
        bind_price = binding_mapping_Direct.loc[binding_mapping_Direct['BINDING::ISBNBindLineExport'] == isbn]['BINDING::DomPrice']
        return(bind_price.to_list()[0])
    except IndexError as ie:
        pass
    
def splitBisac(row):
    bisac_1 = row['TITLE::Bisac1']
    try:
        pieces = bisac_1.split("/")
        top_bisac = pieces[0]
    except AttributeError as ae:
        top_bisac = None
    return top_bisac

def splitCitations(row):
    doi = row['DOI']
    first = doi.split("/")
    second = first[2].split(".")
    title_ID = second[0]
    return title_ID


In [ ]:
# Time stamp for file naming
now = datetime.now()
dt_string = now.strftime("%d%m%Y%H%M%S")


Start by pulling in the sales data received from RT DB

In [ ]:

# Read in All Print and ER sales
PRH_TLT_FY19 = pd.read_csv('data_input/{}'.format(input_file_1)

PRH_TLT_FY20 = pd.read_excel(open('data_input/{}'.format(input_file_2, 'rb'),
              sheet_name='PRH+TLT_FY20') 
PRH_TLT_FY21 = pd.read_excel(open('data_input/{}'.format(input_file_2, 'rb'),
              sheet_name='PRH+TLT_FY21') 
PRH_TLT_FY22 = pd.read_excel(open('data_input/{}'.format(input_file_2, 'rb'),
              sheet_name='PRH+TLT_FY22') 
PRH_TLT_FY23 = pd.read_excel(open('data_input/{}'.format(input_file_2, 'rb'),
              sheet_name='PRH_July-Oct') 


# Read in All EL sales
thirdPartyTransacations_FY20_FY22 = pd.read_excel(open('data_input/{}'.format(input_file_3, 'rb'),
              sheet_name='3rdPartyTransacations') 
thirdPartyTransacations_FY23 = pd.read_excel(open('data_input/{}'.format(input_file_3, 'rb'),
              sheet_name='3rdPartyJuly-Oct')



In [ ]:
# Combine all EL sales into a single DF
all_EL_data = pd.concat([thirdPartyTransacations_FY20_FY22, thirdPartyTransacations_FY23])

# Combine all Print and ER sales into a single DF
all_sales = [PRH_TLT_FY19, PRH_TLT_FY20, PRH_TLT_FY21, PRH_TLT_FY22, PRH_TLT_FY23]
all_PRH_TLT_data = pd.concat(all_sales)




Read and clean EL sales

In [ ]:
# Read e-sales in. 
# Drop empty rows. 
# Drop and rename columns to match print sales. 
all_EL_data.dropna(how="all", inplace=True)
all_EL_data.rename(columns={'isbn': 'ISBN', 'unitssold': 'Units Sold', 'actualamount': 'Amount Sold', 'date': 'Date' }, inplace=True)
all_EL_data['TitleID'] = all_EL_data['TitleID'].astype(str)
all_EL_data['ISBN'] = all_EL_data['ISBN'].astype(str)
all_EL_data['Date'] = pd.to_datetime(all_EL_data.Date)
all_EL_data['Date'] = all_EL_data['Date'].dt.strftime('%m/%Y')

all_EL_data = all_EL_data.groupby(['TitleID', 'ISBN', 'Date']).sum()
all_EL_data.reset_index(inplace=True)


In [ ]:
# Drop any empty rows, convert 'TitleID' and 'PRINT isbn' to a string.
# Group everything by 'ISBN', 'TitleID', and 'Date (month)'
# Rename Print isbn as just 'isbn', which we need to do in order to merge print and electronic sales
all_PRH_TLT_data.dropna(how="all", inplace=True)
all_PRH_TLT_data.rename(columns={'isbn': 'ISBN', 'unitssold': 'Units Sold', 'actualamount': 'Amount Sold', 'date': 'Date' }, inplace=True)
all_PRH_TLT_data['TitleID'] = all_PRH_TLT_data['TitleID'].astype(str)
all_PRH_TLT_data['ISBN'] = all_PRH_TLT_data['ISBN'].astype(str)
all_PRH_TLT_data['Date'] = pd.to_datetime(all_PRH_TLT_data.Date)
all_PRH_TLT_data['Date'] = all_PRH_TLT_data['Date'].dt.strftime('%m/%Y')
all_PRH_TLT_data

Read and clean print sales

In [ ]:
# Divide up print and electronic sales
# Group everything by 'ISBN', 'TitleID', and 'Date (month)'
# Add 'Print' label'
print_sales = all_PRH_TLT_data[all_PRH_TLT_data['revenuetype'] != 'Electronic']
print_sales = print_sales.groupby(['TitleID', 'ISBN', 'Date']).sum()
print_sales.reset_index(inplace=True)
print_sales['Format'] = 'Print'

er_sales = all_PRH_TLT_data[all_PRH_TLT_data['revenuetype'] == 'Electronic']
er_sales = er_sales.groupby(['TitleID', 'ISBN', 'Date']).sum()
er_sales.reset_index(inplace=True)
er_sales['TitleID'] = er_sales.apply(popZeroID, axis=1)


Merge it all together

In [ ]:
# Combine ER and EL sales
#  Add 'Format' label
e_sales = pd.concat([er_sales, all_EL_data])
e_sales['Format'] = 'Electronic'

In [ ]:
# Merge print and electronic sales
merged_sales = pd.concat([print_sales, e_sales])
merged_sales = merged_sales [['TitleID', 'ISBN', 'Date', 'Units Sold', 'Amount Sold', 'Format']]
merged_sales.to_csv('data_output/merged_sales_{}.csv'.format(dt_string), index=None)

Now that we have the data cleaned we need to filter it to only frontlist titles

In [ ]:
s22 = pd.read_csv('data_input/{}'.format(season_list_file))
s22.dropna(subset=['IDTitle'], inplace=True)
s22['IDTitle'] = s22['IDTitle'].astype('str')
s22_tid = s22['IDTitle'].to_list()
# Filter By S22 and F22 titles only
final_df = final_df[final_df['TitleID'].isin(s22_tid)]